In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/df_small.csv')
df.head(3)

,datetime,day_1,day_10,day_11,day_12,day_13,day_14,day_15,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,final_opponent,wrestler
0,200001,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,武蔵丸
1,200001,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,寺尾,闘牙
2,200001,0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,朝乃翔,琴龍
